In [1]:
# diverse module werden zu installieren sein, z.b. pip install psycopg2
# psql (pgAdmin) installieren
# pip install sqlalchemy=1.3.23

from sqlalchemy import create_engine #, MetaData
#from sqlalchemy.orm import declarative_base  # für sqlalchemy 2.0.23
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, DateTime
from sqlalchemy.sql import text

import pandas as pd

In [2]:
# Mit Datenbank verbinden (psql)

import access  # enthält persönliche Zugangsdaten

schema = 'test2' # Datenbankname, Datenbank muß bereits existieren,  z.B. über pgAdmin
host ='127.0.0.1' # localhost
connection_string='postgresql://{}:{}@{}:5432/{}'.format(access.user, access.password, host, schema)
engine = create_engine(connection_string)


In [3]:
##########################################################
# I. Tabellenstruktur über Klassen erzeugen
#   Mit Festlegung von primary_key, Datentypen
#
# Zwei Tabellen werden angelegt:
# 1. Kampagnen: Beihnaltet eine id (Kennung) und Beschreibung
# 2. Messungen: Beinhaltet sämtliche Zeitreihen
# Verknüfung der Messungen mit Kampagnenbeschreibung über Messungen.kampagne und Kampagnen.kennung


Base = declarative_base()

class Kampagnen(Base):
    __tablename__ = 'Kampagnen'

    kennung = Column(Integer, primary_key=True)
    bezeichnung = Column(String(50), nullable=False, unique=True)
    
class Messsungen(Base):
    __tablename__ = 'Messungen'

    kampagne = Column(Integer, nullable=False)
    zeit = Column(String, primary_key=True) # irgendwo primary key setzen
    h_1 = Column(Float)
    h_2 = Column(Float)


Base.metadata.create_all(engine)


In [4]:
# Daten aus Dateien in Datennbanktabellen überführen

# überführe Daten in DataFrames
kf = pd.read_csv("kampagnen.csv")
mf = pd.read_csv("messungen.csv")
mf.head()

,kampagne,zeit,h_1,h_2
0,1,10.10.2023,10,15
1,1,11.10.2023,10,15
2,1,12.10.2023,16,15
3,1,13.10.2023,10,15
4,2,12.11.2023,15,35


In [5]:
# Fülle Datenbanktabelle Kampagnen

con = engine.connect()

for index, row in kf.iterrows():
    kennung, bezeichnung = row['kennung'], row['bezeichnung']
    print(kennung, bezeichnung)
    sql_command = text("""INSERT INTO public."Kampagnen" (kennung,bezeichnung) SELECT {0},'{1}' WHERE NOT EXISTS (SELECT kennung FROM public."Kampagnen" WHERE kennung = {0});""".format(
        kennung, bezeichnung))
    query = con.execute(sql_command)
    #con.commit()  # für sqlalchemy 2.0.23

con.close()



1 erstes mal gemessen
2 wieder was gemessen
3 letzte Messung


In [6]:
# Fülle Daenbanktabelle Messungen

con = engine.connect()

for index, row in mf.iterrows():
    kampagne, zeit, h_1, h_2 = row['kampagne'], row[' zeit'], row[' h_1'], row[' h_2']
    print(kampagne, zeit, h_1, h_2)
    sql_command = text("""INSERT INTO public."Messungen" (kampagne,zeit,h_1,h_2) SELECT {0},'{1}',{2},{3} WHERE NOT EXISTS (SELECT zeit FROM public."Messungen" WHERE zeit = '{1}');""".format(
        kampagne, zeit, h_1, h_2))
    query = con.execute(sql_command)
    #con.commit()  # für sqlalchemy 2.0.23

con.close()


1 10.10.2023 10 15
1 11.10.2023 10 15
1 12.10.2023 16 15
1 13.10.2023 10 15
2 12.11.2023 15 35
2 13.11.2023 10 15
2 14.11.2023 10 45
2 15.11.2023 17 15
2 16.11.2023 10 15
2 17.11.2023 10 15
2 18.11.2023 15 15
2 19.11.2023 10 15
3 13.12.2023 16 35
3 14.12.2023 14 35
3 15.12.2023 20 15
3 16.12.2023 33 15
3 17.12.2023 40 15
3 18.12.2023 13 15
3 19.12.2023 10 15


In [16]:
# Beispielzugriff 1:
# Alle Messungen einer Kampagne

con = engine.connect()

gewaehlte_kampagne=2

sql_command = text("""SELECT * FROM public."Messungen" WHERE kampagne={};""".format(gewaehlte_kampagne))
query = con.execute(sql_command)


for res in query:
    print(res)
    
# für sqlalchemy 2.0.23
#for res in query.all:
#    print(res)

con.close()

(2, '12.11.2023', 15.0, 35.0)
(2, '13.11.2023', 10.0, 15.0)
(2, '14.11.2023', 10.0, 45.0)
(2, '15.11.2023', 17.0, 15.0)
(2, '16.11.2023', 10.0, 15.0)
(2, '17.11.2023', 10.0, 15.0)
(2, '18.11.2023', 15.0, 15.0)
(2, '19.11.2023', 10.0, 15.0)


In [17]:
# Beispielzugriff 2:
# Kampagnenbeschreibung zu einer Messung

con = engine.connect()

gewaehlte_zeit='14.11.2023'  # ist primary key, existiert nur einmal

sql_command = text("""SELECT * FROM public."Messungen" WHERE zeit='{}';""".format(gewaehlte_zeit))
query = con.execute(sql_command)
#print(query.first())
kampagnenkennung = query.first()[0]

sql_command = text("""SELECT bezeichnung FROM public."Kampagnen" WHERE kennung=2;""".format(kampagnenkennung))
query = con.execute(sql_command)
print(query.first())

con.close()

('wieder was gemessen',)


In [18]:
#######################################################################################################
# III. Datenbankzugriffe (psql) mit pandas
# 1. Tabelle lesen


pd.read_sql('Kampagnen', 'postgresql+psycopg2:///{}'.format(schema)) 

,kennung,bezeichnung
0,1,erstes mal gemessen
1,2,wieder was gemessen
2,3,letzte Messung


In [19]:
# 2. Über SQL-Anweisungen

con = engine.connect()

gewaehlte_kampagne=2
fr = pd.read_sql('SELECT * FROM public."Messungen" WHERE kampagne={};'.format(gewaehlte_kampagne), con)
con.close()

fr

,kampagne,zeit,h_1,h_2
0,2,12.11.2023,15.0,35.0
1,2,13.11.2023,10.0,15.0
2,2,14.11.2023,10.0,45.0
3,2,15.11.2023,17.0,15.0
4,2,16.11.2023,10.0,15.0
5,2,17.11.2023,10.0,15.0
6,2,18.11.2023,15.0,15.0
7,2,19.11.2023,10.0,15.0


In [20]:
# 3. Erzeuge Tabelle in Datenbank

data = {'tiefe': [1, 2, 3, 4, 5], 
        'wert': [10, 9.8, 10.1, 12.1, 11.5]} 
  
df = pd.DataFrame(data)

con = engine.connect()

# Erzeuge Tabelle mit den Daten
df.to_sql('Tiefenprofil', con=con, if_exists='replace', 
          index=False) 

con.autocommit = True

con.close()

# Lese aus der Datenbank
#sql1 = '''select * from public."Tiefenprofil";'''
#query = con.execute(sql1)

#print(query.first())
#for res in query:
#    print(res)
#for i in cursor.fetchall(): 
#    print(i) 
  
# conn.commit() 
#conn.close() 

In [21]:
# SQL-Anweisung auf die erzeugte Tabelle
con = engine.connect()
sql_command = """SELECT * FROM public."Tiefenprofil";"""
query = con.execute(sql_command)

for res in query:
    print(res)

(1, 10.0)
(2, 9.8)
(3, 10.1)
(4, 12.1)
(5, 11.5)


In [22]:
engine.dispose()